In [4]:
!pip install vectorbt

  Using cached vectorbt-0.25.5-py3-none-any.whl (526 kB)
  Using cached dateparser-1.2.0-py2.py3-none-any.whl (294 kB)
  Using cached tzlocal-5.2-py3-none-any.whl (17 kB)


# Rebalanceamento de carteira

In [5]:
## Bibliotecas

import numpy    as np
import pandas   as pd
import yfinance as yf
import vectorbt as vbt # biblioteca de backtest

In [9]:
tickers = ['PETR4.SA', 'WEGE3.SA', 'ITSA4.SA', 'BBDC4.SA', 'BRFS3.SA']
start   = '2018-01-01'
end     = '2020-01-01'

# parametrizando 
vbt.settings.array_wrapper['freq'] = 'days' # informando que a periodicidade será em dias
vbt.settings.returns['year_freq']  = '252 days' # informando que teremos 252 dias no ano
vbt.settings.portfolio.stats['incl_unrealized'] = True 

In [10]:
# Obter dados

precos = yf.download(tickers, start = start, end = end)['Close']
precos

[*********************100%***********************]  5 of 5 completed


,BBDC4.SA,BRFS3.SA,ITSA4.SA,PETR4.SA,WEGE3.SA
Date,,,,,
2018-01-02,19.693554,37.270000,8.233388,16.549999,9.623076
2018-01-03,19.773239,37.000000,8.330427,16.700001,9.476923
2018-01-04,20.097670,37.400002,8.487183,16.730000,9.384615
2018-01-05,20.211506,39.240002,8.502112,16.830000,9.423076
2018-01-08,20.205814,39.700001,8.494647,17.030001,9.500000
...,...,...,...,...,...
2019-12-20,26.814425,34.810001,11.372228,30.260000,16.945000
2019-12-23,26.979713,34.529999,11.454338,30.500000,16.895000
2019-12-26,27.317806,35.180000,11.651402,30.910000,17.424999


In [11]:
# Calcular os retornos diários

retornos = precos.pct_change()
retornos

,BBDC4.SA,BRFS3.SA,ITSA4.SA,PETR4.SA,WEGE3.SA
Date,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN
2018-01-03,0.004046,-0.007244,0.011786,0.009064,-0.015188
2018-01-04,0.016408,0.010811,0.018817,0.001796,-0.009740
2018-01-05,0.005664,0.049198,0.001759,0.005977,0.004098
2018-01-08,-0.000282,0.011723,-0.000878,0.011884,0.008163
...,...,...,...,...,...
2019-12-20,-0.020850,0.012802,-0.009299,-0.011434,0.008631
2019-12-23,0.006164,-0.008044,0.007220,0.007931,-0.002951
2019-12-26,0.012531,0.018824,0.017204,0.013443,0.031370


In [12]:
# Pesos

pesos = np.array([0.2, 0.2, 0.2, 0.2, 0.2])
pesos

array([0.2, 0.2, 0.2, 0.2, 0.2])

In [13]:
# retorna o primeiro dia de cada mês no período para verificar a proproção de cada ativo ao longo
# do tempo e se é necessário rebalancear a carteira

precos.index.to_period('m') 

PeriodIndex(['2018-01', '2018-01', '2018-01', '2018-01', '2018-01', '2018-01',
             '2018-01', '2018-01', '2018-01', '2018-01',
             ...
             '2019-12', '2019-12', '2019-12', '2019-12', '2019-12', '2019-12',
             '2019-12', '2019-12', '2019-12', '2019-12'],
            dtype='period[M]', name='Date', length=494)

In [14]:
# Criando uma máscara (vetor) que retorna VERDADEIRO para todo o primeiro dia de cada mês e FALSO para os demais dias
mascara_rb = ~precos.index.to_period('m').duplicated()
mascara_rb

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False,

In [16]:
mascara_rb.shape

(494,)

In [17]:
# construir uma estrutura de dados (array) para colocar os pesos mês a mês
tamanho_rb = np.full_like(precos, np.nan)
tamanho_rb

array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

In [19]:
# este comando atribui ao tamanho_rb a estrutura mascara_rb, sendo que quando mascara_rb = true ele atribui o valor de 'pesos'
tamanho_rb[mascara_rb, :] = pesos 
tamanho_rb

array([[0.2, 0.2, 0.2, 0.2, 0.2],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       ...,
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])

In [ ]:
# Construir o backtest